# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

---

gpt-4o-mini, llama3.2 를 이용한 간단한 프로젝트

In [1]:
# imports
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
import json
from IPython.display import Markdown, display, update_display



In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
if api_key and api_key.startswith('sk-') and len(api_key) > 10:
    print("API key가 괜찮아 보여요")
else:
    print("API key에 문제가 있는거 같은데, 확인해봐야 할듯")

openai = OpenAI()

API key가 괜찮아 보여요


In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
"""

default_question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""


In [5]:
tutor_system_prompt = """You are a helpful coding tutor. 
You have a deep understanding of Python and can explain code precisely and clearly.
This is for a student who is intermediate in Python, and for whom Korean is their first language.
So please answer in Korean.
And to ensure student's knowledge, please ask a follw-up question at the end of your answer.
"""



In [7]:
# Get gpt-4o-mini to answer, with streaming
question = input('코드에 관한 질문을 입력하세요: ')
def get_answer(model=MODEL_GPT, question : str = default_question):
    response = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = [
            {"role": "system", "content": tutor_system_prompt},
            {"role": "user", "content": question}
        ]
        
    )
    result = response.choices[0].message.content
    display(Markdown(result))

get_answer(question)

이 코드는 파이썬의 `yield from` 구문과 집합 표현식(comprehension)을 사용하여 특정 책들의 저자(author) 정보를 생성(generator)하는 역할을 합니다. 자세히 설명하자면:

1. **집합 표현식**:
   ```python
   {book.get("author") for book in books if book.get("author")}
   ```
   이 부분은 `books`라는 리스트(또는 이터러블)에 있는 각 `book` 딕셔너리에서 저자 정보를 추출하고 있습니다. `if book.get("author")` 조건을 사용하여 저자 정보가 존재하는 경우에만 저자 이름을 포함시킵니다. 이렇게 생성된 값들은 중복을 허용하지 않는 집합(set) 형태가 됩니다.

2. **`yield from` 구문**:
   ```python
   yield from { ... }
   ```
   `yield from`은 특정 이터러블(이 경우에는 집합)을 반환하는 역할을 합니다. 이를 통해 이터레이터를 반환하고, 나중에 값을 일일이 리턴하는 대신 더 간단하게 값을 생성할 수 있습니다.

전체적으로 이 코드는 `books` 리스트에서 저자 정보를 추출하여 중복 없이 이터레이터로 생성하는 함수의 일부일 수 있습니다. 이 코드를 사용하는 함수는 호출될 때마다 저자 이름을 하나씩 반환할 수 있게 됩니다.

이해를 돕기 위해, 예를 들어 `books`가 다음과 같은 내용이라고 가정해 봅시다:
```python
books = [
    {"title": "책1", "author": "저자1"},
    {"title": "책2", "author": "저자2"},
    {"title": "책3", "author": None},
    {"title": "책4", "author": "저자1"}
]
```
위의 코드에서는 `저자1`, `저자2`라는 값이 생성되며, `저자1`은 중복이 제거된 상태로 하나만 반환됩니다.

질문: 이 코드에서 `yield from`을 사용하지 않고, 일반 `for` 루프를 사용했다고 가정하면, 코드는 어떻게 달라질까요?

In [37]:
# Get Llama 3.2 to answer
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
def get_answer_llama(question: str = default_question):
    response = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
                {"role": "system", "content": tutor_system_prompt},
                {"role": "user", "content": """
                    Please explain what this code does and why:
                    yield from {book.get("author") for book in books if book.get("author")}
                 
"""},

            ]
    )

    print(response.choices[0].message.content)
get_answer_llama(question)

이 코드는 여러 Books에 대한 정보를 한 sefer에 읽기 더efficient한 방식을 제공한다.

`yield from` keyword는 generator function을return하는 function을return할 때 사용한다. 

이 코드에서는 `books` variable가 List로 구성된 BOOK informações를 담고 있는 list를 제외할 수 있다. 

만약 `book.get("author")`가 None일 경우, 이 코드에서는 해당 book의 정보를 읽지 않는다.

만약 `yield from`를 사용했다면, 이 코드는 Books information을 한 sefer에 읽기 더efficient한 방식을 제공한다.

예를 들어, `books = [{"title": "Book1", "author": "AuthorA"}, {"title": "Book2", "author": "None"}]`

이 경우, `yield from {book.get("author") for book in books if book.get("author")}`는 `["AuthorA", None"]`를 yield한다.

반면, `for book in books: print(book.get("author"))`는 두 번의 읽기를 necessitates한다.

```python
import re

books = [{"title": "Book1", "author": "AuthorA"}, {"title": "Book2", "author": "None"}]

# yield from :效率가 더 good하다 
for info in yield_from ({book.get("author") for book in books if book.get("author")}):
    print(info)

# not yield_from:
for book in books: 
  for author in book.get('author',''): 
     print(author)
```

아래에 두


---

- llama의 답변이 정말 이상한데, 맥북 m1에서도 돌아가는 모델의 한계인거같다.
- 한국어 부분을 뺀 답변

In [8]:
# Get Llama 3.2 to answer
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
def get_answer_llama(question: str = default_question):
    response = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
                {"role": "system", "content": """You are a helpful coding tutor. 
You have a deep understanding of Python and can explain code precisely and clearly.
This is for a student who is intermediate in Python.
To ensure student's knowledge, please ask a follw-up question at the end of your answer.
"""},
                {"role": "user", "content": """
                    Please explain what this code does and why:
                    yield from {book.get("author") for book in books if book.get("author")}
                 
"""},

            ]
    )

    print(response.choices[0].message.content)
get_answer_llama(question)

This code is using Python's generator syntax to extract the "author" values from a list of dictionaries (`books`). Here's how it works:

1. `yield from`: This keyword starts a new generator. It allows you to delegate the generation of values to another iterable (in this case, a dictionary comprehension).
2. `{book.get("author") for book in books if book.get("author")}`: This is a dictionary comprehension that iterates over the list of dictionaries (`books`). For each dictionary, it attempts to get the value of the key `"author"`. If the key exists, its corresponding value is included in the resulting iterator. The `if` condition filters out dictionaries where `"author"` is not present.

So, when we combine these two parts with `yield from`, we create a generator that:

- Yields each `"author"` value found in the list of books.
- Stops iterating over the list of books once it's exhausted, as generators are lazily evaluated.

In essence, this code returns an iterator that produces author

- 영어 답변은 괜찮아보인다